In [1]:
import numpy as np
import pandas as pd
import os
import glob
import requests

In [17]:
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Files", "*.csv"))
num = 0

print(csv_files)

['/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0003682_chromatin_binding_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0005730_nucleolus_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0030312_external_encapsulating_structure_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ Pred_df.csv', '/home/shuting/IDRs/Files/GO0031012_extracellular_matrix_ Tstats_df.csv', '/home/shuting/IDRs/Files/GO0042393_histone_binding_ PostIDR.csv', '/home/shuting/IDRs/Files/GO0042393_histone_binding_ Pred_df.csv',

In [ ]:
# loop over the list of csv files
for f in csv_files:
    
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])
    # print the content
    print('Content:')
    display(df)
    print()
    
    num +=1

In [16]:
num

15

In [18]:
name = "GO0003682_chromatin_binding_ PostIDR.csv"

In [24]:
name_split = name.split()
name_split

['GO0003682_chromatin_binding_', 'PostIDR.csv']

In [25]:
name_split[0]

'GO0003682_chromatin_binding_'

In [61]:
######################## Do not delete ##################################
######################## Example fron ebi API ####################################
############ https://www.ebi.ac.uk/proteins/api/doc/exampleScripts.html ###########
import requests, sys

requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=100&accession=P21802"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
    r.raise_for_status()
    sys.exit()

responseBody = r.json()
print(type(responseBody[0]))

<class 'dict'>


In [2]:
protein_file_name = "../statistics_data/GO0000280_nuclear_division_ Pred_df.csv"
IDR_file_name = "../statistics_data/GO0000280_nuclear_division_ PostIDR.csv"
IDR_cutoff_value1 = 0.5
IDR_cutoff_value2 = 0.6

In [3]:
def create_protein_filter_list(protein_file_name):
    protein_df = pd.read_csv(protein_file_name,  names = ["Protein_name","Score"], header=0, index_col=0)
    # protein_filter = protein_df[protein_df['Score'] > protein_cutoff_value]
    
    # choose top 20 proteins in each pred file
    protein_df["Rank"] = protein_df["Score"].rank(ascending=False) 
    protein_df = protein_df.sort_values("Rank")
    protein_filter = protein_df.head(20)
    return protein_filter

In [4]:
protein_filter = create_protein_filter_list(protein_file_name)
filtered_protein_name = protein_filter.index.values
print(filtered_protein_name)

['Q56NI9' 'Q8WYP5' 'O43663' 'P35251' 'Q69YH5' 'Q99741' 'P46013' 'P10243'
 'P10244' 'Q8NI77' 'Q14207' 'Q96T88' 'O76021' 'Q7RTP6' 'Q14680' 'Q8WWL7'
 'P18583' 'Q9UPQ0' 'Q5UIP0' 'Q86T82']


In [5]:
def compute_IDR_num(filter_IDRs):
    count_dict = dict()
    for ID in filter_IDRs["UniProtID"]:
        if ID in count_dict:
            count_dict[ID] += 1
        else:
            count_dict[ID] = 1

    count_list = []
    for ID in filter_IDRs["UniProtID"]:
        if ID in count_dict:
            count_list.append(count_dict.get(ID))
    filter_IDRs["The number of IDRs in each protein"] = count_list
    return count_list

In [6]:
def get_protein_name(query):
    base_url = "https://www.ebi.ac.uk/proteins/api/{}"
    url = base_url.format(query)
#     print("URL:", url)
    
    try:
        response = requests.get(url, headers={ "Accept" : "application/json"})
#         print("Status code:", response.raise_for_status())  # Check for errors: None == 200
        
        data_list = response.json()
#         print(type(data_list))
        data_dict = data_list[0]
#         print(type(data_dict))
        
        protein_name = data_dict.get('protein').get('recommendedName').get('fullName').get('value')
        return protein_name
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

def get_protein_name_list(uniprot_id_list):
    protein_name_list = []
    for uniprot_id in uniprot_id_list:
        query = "proteins?offset=0&accession=" + uniprot_id
        protein_name = get_protein_name(query)
        protein_name_list.append(protein_name)
    return protein_name_list

In [35]:
# https://plotly.com/python/v3/html-reports/#generate-html-reports-with-d3-graphsusing-python-plotly-and-pandas
def create_IDR_filter_list(IDR_file_name, IDR_cutoff_value1, IDR_cutoff_value2, filtered_protein_name):
    IDR_df = pd.read_csv(IDR_file_name, names = ["Protein IDR","Score"], header=0)
    
    # Split GO_term string in order to let user understand the meaning of each column
    # https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns
    # n=the number of columns you expect
    IDR_df[['UniProtID', 'Redundancy', 'Begin', 'End']] = IDR_df['Protein IDR'].str.split('_', n=4, expand=True)
    
    # combine the "beginning position" and "end position" columns
    IDR_df["The beginning and ending positions of IDR"] = IDR_df['Begin'].astype(str) +"-"+ IDR_df["End"]
    
    
    filter_IDRs = pd.DataFrame()
    for p_name in filtered_protein_name:
        for IDR_name in IDR_df["Protein IDR"].values:
            if p_name in IDR_name:
                s2 = IDR_df[IDR_df["Protein IDR"] == IDR_name]
                filter_IDRs = pd.concat([filter_IDRs, s2], ignore_index=True)
                
    # compute the total numbe of IDRs in each protein            
    count_list = compute_IDR_num(filter_IDRs)
    filter_IDRs["The number of IDRs in each protein"] = count_list
    
    # remove all IDRs whose score <= 0.5
    filter_IDRs = filter_IDRs[filter_IDRs["Score"] > IDR_cutoff_value1]
    
    # remove useless columns
    filter_IDRs = filter_IDRs.drop(columns=['Protein IDR', 'Redundancy', 'Begin', 'End'])
    
    # rename columns in order to make them more understandable
    filter_IDRs = filter_IDRs[['UniProtID', 'The beginning and ending positions of IDR', 'Score', 'The number of IDRs in each protein']]
    
    # if the protein has more than 2 IDRs, remove IDRs whose score <= 0.6
    temp_df = filter_IDRs[filter_IDRs["The number of IDRs in each protein"]>2]
    filter_IDRs = filter_IDRs.drop((temp_df["Score"] < 0.6).index.values)
    
    # hold two digits after the decimal points
#     filter_IDRs["Score"] = filter_IDRs["Score"].astype(float)
#     filter_IDRs["Score"] = filter_IDRs["Score"].round(2)
    
    # rename "Score"
#     filter_IDRs = filter_IDRs.rename(columns={"Score": "The probability of the association between the IDR and the function"})
    
    # remove the "Score" column since their probabilities are amostly 1, so not very informative/interesting
    filter_IDRs = filter_IDRs.drop(columns=['Score'])
    
    # add protein name column
    uniprot_id_list = filter_IDRs["UniProtID"]
    protein_name_list = get_protein_name_list(uniprot_id_list)
    filter_IDRs.insert(1, "Protein name", protein_name_list)

    html_filter_IDRs = filter_IDRs.to_html(index=False).replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    html_filter_IDRs = html_filter_IDRs.replace('<td>', '<td style="text-align: center;">') # center text
    html_filter_IDRs= html_filter_IDRs.replace('<th>', '<th style="text-align: center;">') # center column names
    return filter_IDRs

In [36]:
filter_IDRs = create_IDR_filter_list(IDR_file_name, IDR_cutoff_value1, IDR_cutoff_value2, filtered_protein_name)
filter_IDRs

,UniProtID,Protein name,The beginning and ending positions of IDR,The number of IDRs in each protein
0,Q56NI9,N-acetyltransferase ESCO2,1-370,1
1,Q8WYP5,Protein ELYS,1008-2266,1
2,O43663,Protein regulator of cytokinesis 1,441-620,1
6,Q69YH5,Cell division cycle-associated protein 2,1-1023,1
7,Q99741,Cell division control protein 6 homolog,1-152,1
8,P46013,Proliferation marker protein Ki-67,98-3256,1
12,P10244,Myb-related protein B,178-658,1
13,Q8NI77,Kinesin-like protein KIF18A,617-898,1
14,Q14207,Protein NPAT,72-1411,1
15,Q96T88,E3 ubiquitin-protein ligase UHRF1,593-675,2
